# Day 1: Launchpad - Colab Workflow, Gemini Vibe-Coding, EDA, and Splitting Correctly

**MGMT 47400 - Predictive Analytics**  
**4-Week Online Course**  
**Day 1 - Tuesday May 18, 2027**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/blob/main/notebooks/01_launchpad_eda_splits.ipynb)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Operate course workflow in Google Colab (run-all, save-copy, submit link)
2. Use Gemini in Colab to accelerate coding while preserving accountability (explain + verify)
3. Perform structured EDA (types, missingness, target distribution, leakage sniff test)
4. Create train/validation/test splits with reproducible seeds
5. Identify obvious leakage patterns before modeling

---

## 1. Setup: Installs, Imports, Seeds, Display Settings

First, let's set up our environment with all necessary packages and configurations.

In [ ]:
# Install required packages (uncomment if needed)
# !pip install pandas numpy matplotlib seaborn scikit-learn --quiet

# Core imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings

# Display settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 3)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print("✓ Setup complete!")
print(f"Random seed: {RANDOM_SEED}")

## 2. Gemini Workflow Rules: Ask → Verify → Document

### Using Gemini Responsibly in This Course

**The "Ask → Verify → Document" Pattern:**

1. **Ask**: Use Gemini to draft code or explain concepts
2. **Verify**: Run the code, check outputs, understand what it does
3. **Document**: Add comments explaining the logic in your own words

**What's allowed:**
- Using Gemini to generate boilerplate code
- Asking Gemini to explain error messages
- Getting suggestions for visualizations or analyses

**What's required:**
- You must understand every line of code you submit
- You must verify that generated code works correctly
- You must add your own comments and documentation

**Accountability:**
- You are responsible for all code you submit, even if Gemini generated it
- Be prepared to explain your code and methodology
- If you can't explain it, you don't understand it well enough yet

---

## 3. Load Dataset

We'll use a sample dataset to practice our EDA and splitting workflow.

In [ ]:
# For demonstration, we'll use the California Housing dataset
from sklearn.datasets import fetch_california_housing

# Load data
california = fetch_california_housing(as_frame=True)
df = california.frame

print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
df.head()

## 4. EDA Checklist: Types, Missingness, Target Distribution

### 4.1 Data Types Audit

In [ ]:
# Check data types
print("=== DATA TYPES ===")
print(df.dtypes)
print("\n=== DATA INFO ===")
df.info()

### 4.2 Missingness Check

In [ ]:
# Check for missing values
missing = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing_Count': missing,
    'Missing_Percentage': missing_pct
})

print("=== MISSING VALUES ===")
print(missing_df[missing_df['Missing_Count'] > 0])

if missing.sum() == 0:
    print("✓ No missing values detected!")

### 4.3 Basic Descriptive Statistics

In [ ]:
# Descriptive statistics
print("=== DESCRIPTIVE STATISTICS ===")
df.describe()

### 4.4 Target Distribution

In [ ]:
# Visualize target distribution
target_col = 'MedHouseVal'

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(df[target_col], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel(target_col)
axes[0].set_ylabel('Frequency')
axes[0].set_title('Target Distribution (Histogram)')
axes[0].axvline(df[target_col].mean(), color='red', linestyle='--', label=f'Mean: {df[target_col].mean():.2f}')
axes[0].legend()

# Box plot
axes[1].boxplot(df[target_col])
axes[1].set_ylabel(target_col)
axes[1].set_title('Target Distribution (Box Plot)')

plt.tight_layout()
plt.show()

print(f"Target statistics:")
print(f"  Mean: {df[target_col].mean():.3f}")
print(f"  Median: {df[target_col].median():.3f}")
print(f"  Std: {df[target_col].std():.3f}")
print(f"  Min: {df[target_col].min():.3f}")
print(f"  Max: {df[target_col].max():.3f}")

### 4.5 Feature Distributions

In [ ]:
# Visualize feature distributions
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Plot histograms for all numeric features
n_cols = 3
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 4))
axes = axes.flatten()

for idx, col in enumerate(numeric_cols):
    axes[idx].hist(df[col], bins=30, edgecolor='black', alpha=0.7)
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Frequency')
    axes[idx].set_title(f'{col} Distribution')

# Hide extra subplots
for idx in range(len(numeric_cols), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

### 4.6 Correlation Analysis

In [ ]:
# Correlation matrix
corr_matrix = df.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

# Correlations with target
print("\n=== CORRELATIONS WITH TARGET ===")
target_corr = corr_matrix[target_col].sort_values(ascending=False)
print(target_corr)

## 📝 PAUSE-AND-DO Exercise 1 (10 minutes)

**Task:** Complete the EDA checklist on the dataset and summarize 3 key findings.

**Instructions:**
1. Review all the EDA outputs above
2. Identify 3 important patterns or insights
3. Write your findings in the cell below

**What to look for:**
- Unusual distributions
- Strong correlations
- Potential data quality issues
- Features that might be useful predictors

---

### YOUR FINDINGS HERE:

**Finding 1:**  
[Your observation]

**Finding 2:**  
[Your observation]

**Finding 3:**  
[Your observation]

---

## 5. Train/Validation/Test Splits + Leakage Sniff Test

### 5.1 Why Three Splits?

- **Train set**: Used to fit models
- **Validation set**: Used to tune hyperparameters and compare models
- **Test set**: Final "lockbox" evaluation - touch only once at the end

### 5.2 Creating Reproducible Splits

In [ ]:
# Separate features and target
X = df.drop(columns=[target_col])
y = df[target_col]

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Split: 60% train, 20% validation, 20% test
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.20, random_state=RANDOM_SEED
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=RANDOM_SEED  # 0.25 * 0.80 = 0.20
)

print("\n=== SPLIT SIZES ===")
print(f"Train: {len(X_train)} samples ({len(X_train)/len(df)*100:.1f}%)")
print(f"Validation: {len(X_val)} samples ({len(X_val)/len(df)*100:.1f}%)")
print(f"Test: {len(X_test)} samples ({len(X_test)/len(df)*100:.1f}%)")
print(f"Total: {len(X_train) + len(X_val) + len(X_test)} samples")

### 5.3 Split Sanity Checks

In [ ]:
# Check target distribution across splits
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, (split_name, split_y) in enumerate([('Train', y_train), ('Validation', y_val), ('Test', y_test)]):
    axes[idx].hist(split_y, bins=30, edgecolor='black', alpha=0.7)
    axes[idx].set_xlabel(target_col)
    axes[idx].set_ylabel('Frequency')
    axes[idx].set_title(f'{split_name} Set (n={len(split_y)})')
    axes[idx].axvline(split_y.mean(), color='red', linestyle='--', label=f'Mean: {split_y.mean():.2f}')
    axes[idx].legend()

plt.tight_layout()
plt.show()

print("=== TARGET DISTRIBUTION ACROSS SPLITS ===")
print(f"Train mean: {y_train.mean():.3f}, std: {y_train.std():.3f}")
print(f"Val mean: {y_val.mean():.3f}, std: {y_val.std():.3f}")
print(f"Test mean: {y_test.mean():.3f}, std: {y_test.std():.3f}")

### 5.4 Leakage Sniff Test

**Common leakage patterns to watch for:**

1. **Target leakage**: Features that contain information from the future or are derived from the target
2. **Test set contamination**: Using test data to make preprocessing or modeling decisions
3. **Time-based leakage**: Using future data to predict the past
4. **Preprocessing leakage**: Fitting transformers on the full dataset before splitting

In [ ]:
# Leakage checklist
print("=== LEAKAGE SNIFF TEST ===")
print("\n✓ Checklist:")
print("  [✓] Split data before any preprocessing")
print("  [✓] No overlap between train/val/test indices")
print("  [ ] Check feature descriptions for suspicious variables")
print("  [ ] Verify no future information in features")
print("  [ ] Confirm target is not derived from features")

print("\n=== FEATURE NAMES ===")
print("Review these carefully for potential leakage:")
for col in X.columns:
    print(f"  - {col}")

print("\n⚠️ Red flags to watch for:")
print("  - Features with 'target', 'outcome', 'label' in name")
print("  - Perfect correlations (r > 0.99)")
print("  - Features that would not be available at prediction time")
print("  - ID columns that encode information about the target")

## 📝 PAUSE-AND-DO Exercise 2 (10 minutes)

**Task:** Create train/val/test splits and write 3 leakage risks specific to this dataset.

**Instructions:**
1. The splits are already created above - verify they look reasonable
2. Review the feature names and descriptions
3. Identify 3 potential leakage risks for this type of problem
4. Write your analysis below

---

### YOUR LEAKAGE ANALYSIS HERE:

**Potential Risk 1:**  
[Describe a feature or scenario that could cause leakage]

**Potential Risk 2:**  
[Describe another leakage risk]

**Potential Risk 3:**  
[Describe a third leakage risk]

---

## 6. Wrap-Up: Key Takeaways

### What We Learned Today:

1. **Colab Workflow**: How to use Google Colab for course assignments
2. **Gemini Guidelines**: Ask → Verify → Document pattern for responsible AI assistance
3. **EDA Essentials**: Data types, missingness, distributions, correlations
4. **Proper Splitting**: Train/val/test splits with reproducible seeds
5. **Leakage Awareness**: Common patterns to watch for

### Next-Day Readiness:

- ✓ You can load and explore a dataset
- ✓ You can create proper train/val/test splits
- ✓ You can identify potential leakage risks
- ✓ You're ready for Day 2: Preprocessing Pipelines

### Remember:

> **"Split first, preprocess second, model third"**  
> This order prevents leakage and ensures valid evaluation.

---

## 7. Submission Instructions

### To Submit This Notebook:

1. **Run All Cells**: Execute `Runtime → Run all` to ensure everything works
2. **Save a Copy**: `File → Save a copy in Drive`
3. **Get Shareable Link**: Click `Share` and set to "Anyone with the link can view"
4. **Submit Link**: Paste the link in the LMS assignment

### Before Submitting, Check:

- [ ] All cells execute without errors
- [ ] All outputs are visible
- [ ] Exercise responses are complete
- [ ] Notebook is shared with correct permissions

---

## Bibliography

- James, G., Witten, D., Hastie, T., & Tibshirani, R. (2021). *An Introduction to Statistical Learning with Python* (ISLP). Springer.
- scikit-learn User Guide: [Cross-validation](https://scikit-learn.org/stable/modules/cross_validation.html)
- scikit-learn User Guide: [Common pitfalls and recommended practices](https://scikit-learn.org/stable/common_pitfalls.html)
- Kaggle Learn: [Data Leakage](https://www.kaggle.com/learn/data-leakage)

---

**End of Day 1 Notebook** 🎉